### Convert BC Shelter Addresses JSON to CSV

In [3]:
import pandas as pd
import json

json_path = "data/bc_shelter_addresses.json"
with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Convert to DataFrame
df = pd.json_normalize(data)
address_column = df.loc[:, ['address_text']] if 'address_text' in df.columns else df.iloc[:, [0]]
address_column.columns = ['addressString']
# Save to CSV
csv_path = "data/bc_shelter_addresses.csv"
address_column.to_csv(csv_path, index=False, encoding='utf-8')

### Clean BC Shelter Addresses with PostalCode

In [ ]:
import pandas as pd
import urllib.parse
import urllib.request
import xml.dom.minidom
import time

# --- Canada Post AddressComplete API functions ---
def find_address(Key, SearchTerm):
    base_url = "http://ws1.postescanada-canadapost.ca/AddressComplete/Interactive/Find/v2.10/xmla.ws?"
    params = {
        "Key": Key,
        "SearchTerm": SearchTerm,
        "Country": "CAN",
        "LanguagePreference": "EN",
        "MaxSuggestions": "1",
        "MaxResults": "1"
    }
    url = base_url + urllib.parse.urlencode(params)
    with urllib.request.urlopen(url) as response:
        xml_data = response.read()

    doc = xml.dom.minidom.parseString(xml_data)
    data_nodes = doc.getElementsByTagName("Row")

    if not data_nodes:
        return None

    id = data_nodes[0].getAttribute("Id")
    next_action = data_nodes[0].getAttribute("Next")

    # If 'Next' says 'Find', do another query
    while next_action == "Find":
        params = {
            "Key": Key,
            "SearchTerm": "",
            "LastId": id,
            "Country": "CAN",
            "LanguagePreference": "EN",
            "MaxSuggestions": "1",
            "MaxResults": "1"
        }
        url = base_url + urllib.parse.urlencode(params)
        with urllib.request.urlopen(url) as response:
            xml_data = response.read()

        doc = xml.dom.minidom.parseString(xml_data)
        data_nodes = doc.getElementsByTagName("Row")

        if not data_nodes:
            return None

        id = data_nodes[0].getAttribute("Id")
        next_action = data_nodes[0].getAttribute("Next")

    return id

def retrieve_address(Key, Id):
    base_url = "http://ws1.postescanada-canadapost.ca/AddressComplete/Interactive/Retrieve/v2.11/xmla.ws?"
    params = {
        "Key": Key,
        "Id": Id
    }
    url = base_url + urllib.parse.urlencode(params)
    with urllib.request.urlopen(url) as response:
        xml_data = response.read()
    doc = xml.dom.minidom.parseString(xml_data)
    schema_nodes = doc.getElementsByTagName("Column")
    data_nodes = doc.getElementsByTagName("Row")
    if not data_nodes:
        return None
    row = {
        col.getAttribute("Name"): data_nodes[0].getAttribute(col.getAttribute("Name"))
        for col in schema_nodes
    }
    return row.get("PostalCode", "")

# --- Step 1: Load and filter data ---
df = pd.read_csv("data/job-784641-result-1.csv")
filtered_df = df[df["precisionPoints"] >= 99].copy()
filtered_df = filtered_df.drop_duplicates(subset=["fullAddress"])

print(f"Filtered and deduplicated rows: {len(filtered_df)}")

filtered_df["civicNumber"] = filtered_df["civicNumber"].apply(
    lambda x: str(int(x)) if pd.notnull(x) and str(x).replace('.', '', 1).isdigit() else ""
)

#--- Step 2: Use AddressComplete API to get postal codes ---

#Sample size for testing
sample_size = 5  # change to X number or another number as needed
sample_df = filtered_df.sample(n=sample_size, random_state=42).copy()

display(sample_df)

api_key = "BK53-ME26-ZP55-XK78"
postal_codes = []

for addr in sample_df["fullAddress"]:
    try:
        id = find_address(api_key, addr)
        if id:
            postal = retrieve_address(api_key, id)
        else:
            postal = ""
    except Exception as e:
        postal = ""
    postal_codes.append(postal)
    time.sleep(0.2)  

sample_df["postalCode"] = postal_codes

# --- Step 3: Format final output ---
formatted_df = sample_df[[
    "unitNumber",
    "civicNumber",
    "streetName",
    "streetType",
    "streetDirection",
    "provinceCode",
    "localityName",
    "postalCode",
    "matchPrecision",
    "precisionPoints"
]].copy()

# --- Step 4: Export to Cleaned CSV ---
formatted_df.to_csv("data/bc_shelter_addresses_cleaned.csv", index=False)
print("File saved as: bc_shelter_addresses_cleaned.csv")
display(formatted_df.head(5))

Filtered and deduplicated rows: 201


,sequenceNumber,resultNumber,yourId,fullAddress,intersectionName,score,matchPrecision,precisionPoints,faults,siteName,...,intersectionID,fullSiteDescriptor,accessNotes,siteStatus,siteRetireDate,changeDate,isOfficial,degree,executionTime,sid
150,151,1,NaN,"268 Fulford-Ganges Rd, Salt Spring Island, BC",NaN,99,CIVIC_NUMBER,100,[PROVINCE.missing:1],NaN,...,NaN,NaN,"""",active,9999-12-31,2025-02-07,True,NaN,0.437,2892975.0
28,29,1,NaN,"108 E Hastings St, Vancouver, BC",NaN,99,CIVIC_NUMBER,100,[PROVINCE.missing:1],NaN,...,NaN,NaN,"""",active,9999-12-31,2025-02-07,True,NaN,1.443,4458023.0
50,51,1,NaN,"1240 Yates St, Victoria, BC",NaN,99,CIVIC_NUMBER,100,[PROVINCE.missing:1],NaN,...,NaN,NaN,"""",active,9999-12-31,2025-02-07,True,NaN,0.063,3039547.0
249,250,1,NaN,"5653 Wharf Ave, Sechelt, BC",NaN,99,CIVIC_NUMBER,100,[PROVINCE.missing:1],NaN,...,NaN,NaN,"""",active,9999-12-31,2025-02-07,True,NaN,0.083,4330127.0
203,204,1,NaN,"390 Main St, Vancouver, BC",NaN,99,CIVIC_NUMBER,100,[PROVINCE.missing:1],NaN,...,NaN,NaN,"""",active,9999-12-31,2025-02-07,True,NaN,0.084,3673792.0


File saved as: bc_shelter_addresses_cleaned.csv


,unitNumber,civicNumber,streetName,streetType,streetDirection,province,localityName,postalCode,matchPrecision,precisionPoints
150,NaN,268,Fulford-Ganges,Rd,NaN,BC,Salt Spring Island,V8K 2K6,CIVIC_NUMBER,100
28,NaN,108,Hastings,St,E,BC,Vancouver,V6A 0H1,CIVIC_NUMBER,100
50,NaN,1240,Yates,St,NaN,BC,Victoria,V8V 3N3,CIVIC_NUMBER,100
249,NaN,5653,Wharf,Ave,NaN,BC,Sechelt,V7Z 0H5,CIVIC_NUMBER,100
203,NaN,390,Main,St,NaN,BC,Vancouver,V6A 2T1,CIVIC_NUMBER,100


### Clean BC Patient Addresses with PostalCode

In [42]:
import pandas as pd
import urllib.parse
import urllib.request
import xml.dom.minidom
import time

# --- Canada Post AddressComplete API functions ---
def find_address(Key, SearchTerm):
    base_url = "http://ws1.postescanada-canadapost.ca/AddressComplete/Interactive/Find/v2.10/xmla.ws?"
    params = {
        "Key": Key,
        "SearchTerm": SearchTerm,
        "Country": "CAN",
        "LanguagePreference": "EN",
        "MaxSuggestions": "1",
        "MaxResults": "1"
    }
    url = base_url + urllib.parse.urlencode(params)
    with urllib.request.urlopen(url) as response:
        xml_data = response.read()

    doc = xml.dom.minidom.parseString(xml_data)
    data_nodes = doc.getElementsByTagName("Row")

    if not data_nodes:
        return None

    id = data_nodes[0].getAttribute("Id")
    next_action = data_nodes[0].getAttribute("Next")

    # If 'Next' says 'Find', do another query
    while next_action == "Find":
        params = {
            "Key": Key,
            "SearchTerm": "",
            "LastId": id,
            "Country": "CAN",
            "LanguagePreference": "EN",
            "MaxSuggestions": "1",
            "MaxResults": "1"
        }
        url = base_url + urllib.parse.urlencode(params)
        with urllib.request.urlopen(url) as response:
            xml_data = response.read()

        doc = xml.dom.minidom.parseString(xml_data)
        data_nodes = doc.getElementsByTagName("Row")

        if not data_nodes:
            return None

        id = data_nodes[0].getAttribute("Id")
        next_action = data_nodes[0].getAttribute("Next")

    return id

def retrieve_address(Key, Id):
    base_url = "http://ws1.postescanada-canadapost.ca/AddressComplete/Interactive/Retrieve/v2.11/xmla.ws?"
    params = {
        "Key": Key,
        "Id": Id
    }
    url = base_url + urllib.parse.urlencode(params)
    with urllib.request.urlopen(url) as response:
        xml_data = response.read()
    doc = xml.dom.minidom.parseString(xml_data)
    schema_nodes = doc.getElementsByTagName("Column")
    data_nodes = doc.getElementsByTagName("Row")
    if not data_nodes:
        return None
    row = {
        col.getAttribute("Name"): data_nodes[0].getAttribute(col.getAttribute("Name"))
        for col in schema_nodes
    }
    return row.get("PostalCode", "")

# --- Step 1: Load and filter data ---
df = pd.read_csv("data/job-785055-result-1.csv")
filtered_df = df[df["precisionPoints"] >= 99].copy()
filtered_df = filtered_df.drop_duplicates(subset=["fullAddress"])

print(f"Filtered and deduplicated rows: {len(filtered_df)}")

filtered_df["civicNumber"] = filtered_df["civicNumber"].apply(
    lambda x: str(int(x)) if pd.notnull(x) and str(x).replace('.', '', 1).isdigit() else ""
)

#--- Step 2: Use AddressComplete API to get postal codes ---

#Sample size for testing
# sample_size = 5  # change to X number or another number as needed
# sample_df = filtered_df.sample(n=sample_size, random_state=42).copy()

sample_df = filtered_df.copy()

display(sample_df)

api_key = "BK53-ME26-ZP55-XK78"
postal_codes = []

for addr in sample_df["fullAddress"]:
    try:
        id = find_address(api_key, addr)
        if id:
            postal = retrieve_address(api_key, id)
        else:
            postal = ""
    except Exception as e:
        postal = ""
    postal_codes.append(postal)
    time.sleep(0.2)  

sample_df["postalCode"] = postal_codes

# --- Step 3: Format final output ---
formatted_df = sample_df[[
    "unitNumber",
    "civicNumber",
    "streetName",
    "streetType",
    "streetDirection",
    "provinceCode",
    "localityName",
    "postalCode",
    "matchPrecision",
    "precisionPoints"
]].copy()

# --- Step 4: Export to Cleaned CSV ---
formatted_df.to_csv("data/bc_patient_addresses_cleaned.csv", index=False)
print("File saved as: bc_patient_addresses_cleaned.csv")
display(formatted_df.head(5))

Filtered and deduplicated rows: 5


,sequenceNumber,resultNumber,yourId,fullAddress,intersectionName,score,matchPrecision,precisionPoints,faults,siteName,...,intersectionID,fullSiteDescriptor,accessNotes,siteStatus,siteRetireDate,changeDate,isOfficial,degree,executionTime,sid
0,1,1,NaN,"1000 Burrard St, Vancouver, BC",NaN,52,CIVIC_NUMBER,100,"[POSTAL_ADDRESS_ELEMENT.notAllowed:1, LOCALITY...",NaN,...,NaN,NaN,"""",active,9999-12-31,2025-02-07,True,NaN,21.479,1143135
1,2,1,NaN,"1014 Homer St, Vancouver, BC",NaN,52,CIVIC_NUMBER,100,"[POSTAL_ADDRESS_ELEMENT.notAllowed:1, LOCALITY...",NaN,...,NaN,NaN,"""",active,9999-12-31,2025-02-07,True,NaN,17.180,4062562
2,3,1,NaN,"1016 Alberni St, Vancouver, BC",NaN,52,CIVIC_NUMBER,100,"[POSTAL_ADDRESS_ELEMENT.notAllowed:1, LOCALITY...",NaN,...,NaN,NaN,"""",active,9999-12-31,2025-02-07,True,NaN,26.583,1553052
3,4,1,NaN,"1018 Granville St, Vancouver, BC",NaN,52,CIVIC_NUMBER,100,"[POSTAL_ADDRESS_ELEMENT.notAllowed:1, LOCALITY...",NaN,...,NaN,NaN,"""",active,9999-12-31,2025-02-07,True,NaN,55.047,4462071
4,5,1,NaN,"1240 Yates St, Victoria, BC",NaN,52,CIVIC_NUMBER,100,"[POSTAL_ADDRESS_ELEMENT.notAllowed:1, LOCALITY...",NaN,...,NaN,NaN,"""",active,9999-12-31,2025-02-07,True,NaN,7.792,3039547


File saved as: bc_patient_addresses_cleaned.csv


,unitNumber,civicNumber,streetName,streetType,streetDirection,provinceCode,localityName,postalCode,matchPrecision,precisionPoints
0,NaN,1000,Burrard,St,NaN,BC,Vancouver,V6Z 2R9,CIVIC_NUMBER,100
1,NaN,1014,Homer,St,NaN,BC,Vancouver,V6B 2W9,CIVIC_NUMBER,100
2,NaN,1016,Alberni,St,NaN,BC,Vancouver,V6E 1A3,CIVIC_NUMBER,100
3,NaN,1018,Granville,St,NaN,BC,Vancouver,V6Z 1L5,CIVIC_NUMBER,100
4,NaN,1240,Yates,St,NaN,BC,Victoria,V8V 3N3,CIVIC_NUMBER,100


### Verify IsNonMarket Addresses

In [44]:
patient_df = pd.read_csv("data/bc_patient_addresses_cleaned.csv")
shelter_df = pd.read_csv("data/bc_shelter_addresses_cleaned.csv")

def build_full_address_string(df):
    return (
        df["unitNumber"].fillna("").astype(str).str.strip() + " " +
        df["civicNumber"].fillna("").astype(str).str.strip() + " " +
        df["streetDirection"].fillna("").astype(str).str.strip() + " " +
        df["streetName"].fillna("").astype(str).str.strip() + " " +
        df["streetType"].fillna("").astype(str).str.strip() + ", " +
        df["localityName"].fillna("").astype(str).str.strip() + ", " +
        df["provinceCode"].fillna("").astype(str).str.strip() + ", " +
        df["postalCode"].fillna("").astype(str).str.strip()
    ).str.replace(" +", " ", regex=True).str.lower().str.strip()

# Generate address strings
patient_df["addressString"] = build_full_address_string(patient_df)
shelter_df["addressString"] = build_full_address_string(shelter_df)

# Compare and label matches
shelter_set = set(shelter_df["addressString"])
patient_df["Match"] = patient_df["addressString"].apply(lambda x: "Yes" if x in shelter_set else "No")

# Save to CSV
patient_df.to_csv("data/isnonmarket_addresses.csv", index=False)
print("File saved as: isnonmarket_addresses.csv")
display(patient_df.head(5))

File saved as: isnonmarket_addresses.csv


,unitNumber,civicNumber,streetName,streetType,streetDirection,provinceCode,localityName,postalCode,matchPrecision,precisionPoints,addressString,Match
0,NaN,1000,Burrard,St,NaN,BC,Vancouver,V6Z 2R9,CIVIC_NUMBER,100,"1000 burrard st, vancouver, bc, v6z 2r9",No
1,NaN,1014,Homer,St,NaN,BC,Vancouver,V6B 2W9,CIVIC_NUMBER,100,"1014 homer st, vancouver, bc, v6b 2w9",No
2,NaN,1016,Alberni,St,NaN,BC,Vancouver,V6E 1A3,CIVIC_NUMBER,100,"1016 alberni st, vancouver, bc, v6e 1a3",No
3,NaN,1018,Granville,St,NaN,BC,Vancouver,V6Z 1L5,CIVIC_NUMBER,100,"1018 granville st, vancouver, bc, v6z 1l5",No
4,NaN,1240,Yates,St,NaN,BC,Victoria,V8V 3N3,CIVIC_NUMBER,100,"1240 yates st, victoria, bc, v8v 3n3",Yes
